# Scraping many pages + Using Selenium

## The pages we'll be looking at

If I wanted to read specific information about a specfic mine, it takes a few steps. **Do these steps with your browser before you try any programming.**

1. Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm)
2. Scroll down to **Mine Identification Number (ID) Search**
3. Type in a mine ID number, such as `3503598`, click **Search**
4. I'm on a page! It lists the MINE NAME and MINE OWNER.

After searching for and finding a mine, I can use this page to **find reports about this mine**. Some of the reports are on accidents, violations, inspections, health samples and more. To get those reports:

1. Search for a mine (if you haven't already)
2. Scroll down and change **Beginning Date** to `1/1/1995` (violation reports begin in 1995, accidents begin in 1983)
3. Select the report type of `Violations`
4. Click **Get Report**
5. I'm on a page! It lists ALL OF THE MINE'S VIOLATIONS.

By changing the report type you're searching for you can find all sorts of different data.

# Researching mine information

## Preparation 

### When you search for information on a specific mine, what URL should Selenium visit first?

- *TIP: the answer is NOT `https://arlweb.msha.gov/drs/ASP/BasicMineInfonew.asp`*

In [143]:
# https://arlweb.msha.gov/drs/drshome.htm

### How can you identify the text field we're going to type the Mine ID into?

Selenium can find elements by:

- name
- Class
- ID
- CSS selector (**ASK ME WHAT THIS IS** if you don't know)
- XPath (**ASK ME WHAT THIS IS** because you definitely don't know)
- Link text
- Partial link text

So in other words, what's unique about this element?

In [144]:
# //*[@id="inputdrs"]

### How can you identify the search button we're going to click, or the form we're going to submit?

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

In [145]:
# The button's xpath is:
#//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input

### Use Selenium to search using the mine ID `3901432`. Get me the operator's name by scraping.

- *TIP: You can find elements/text using Selenium, or use BeautifulSoup with `doc = BeautifulSoup(driver.page_source)`*

In [146]:
from selenium import webdriver

In [147]:
driver = webdriver.Chrome()

In [148]:
driver.get("https://arlweb.msha.gov/drs/drshome.htm")

In [149]:
input_field = driver.find_element_by_name("MineId")
input_field

<selenium.webdriver.remote.webelement.WebElement (session="d28ce10f3c02c47146758e12eaf484be", element="0.6019398950913999-1")>

In [150]:
input_field.send_keys("3901432")

In [151]:
search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
search_button.click()

In [152]:
name_tag = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[1]/tbody/tr[3]/td[2]/font/b')

In [153]:
name_tag.text

'Krueger Brothers Gravel & Dirt'

# Using .apply to find data about SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

In [154]:
import pandas as pd
df = pd.read_csv("mines-subset.csv", dtype={'id': 'str'})
df.head()

,id
0,4104757
1,0801306
2,3609931


### Open up `mines-subset.csv` in a text editor, then look at your dataframe. Is something different about them?

In [155]:
!cat mines-subset.csv

id
4104757
0801306
3609931

### Scrape the operator's name for each of those mines and print it

- *TIP: use .apply and a function*
- *TIP: If you need help with .apply, look at the "Using apply in pandas" notebook *

In [156]:
def scrape_name(row):
    driver.get("https://arlweb.msha.gov/drs/drshome.htm")
    input_field = driver.find_element_by_name("MineId")
    input_field.send_keys(row['id'])
    search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
    search_button.click()
    name_tag = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[1]/tbody/tr[3]/td[2]/font/b')
    return name_tag.text
    #print(name_tag.text)
    #print(row['id'])
    #print("-----")
    #driver = webdriver.Chrome()
df['name'] = df.apply(scrape_name, axis=1)
#driver.close()

In [157]:
df.head()

,id,name
0,4104757,Dirt Works
1,0801306,"Holley Dirt Company, Inc"
2,3609931,M.R. Dirt Inc.


### Scrape the operator's name and save it into a new column

- *TIP: Use .apply and a function*
- *TIP: Remember to use `return`*

In [158]:
df['name'] = df.apply(scrape_name, axis=1)

In [159]:
df.head()

,id,name
0,4104757,Dirt Works
1,0801306,"Holley Dirt Company, Inc"
2,3609931,M.R. Dirt Inc.


# Researching mine violations

Read the very top again to remember how to find mine violations

### When you search for a mine's violations, what URL is Selenium going to start on?

- *TIP: `requests` can send form data to load in the middle of a bunch of steps, but Selenium has to start at the beginning

In [160]:
# url = https://arlweb.msha.gov/drs/drshome.htm

### When you're searching for violations from the Mine Information page, how are you going to identify the "Beginning Date" field?

In [161]:
# //*[@id="content"]/form[1]/table[2]/tbody/tr[2]/td/font/input[1]

### When you're searching for violations from the Mine Information page, how are you going to identify the "Violations" button?

In [162]:
# //*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input

### When you're searching for violations from the Mine Information page, how are you going to identify the form or the button to click to get a list of the violations?

In [163]:
# //*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input

### Using the mine ID `3901432`, scrape all of their violations since 1/1/1995

**Save this into a CSV called `3901432-violations.csv`.** This CSV must include the following fields:

- Citation number
- Case number
- Standard violated
- Link to standard
- Proposed penalty
- Amount paid to date

**Tips:**

- *TIP: It's probably worth it to print them all first, then save them to a CSV once you know it's all working.*
- *TIP: You'll use the parent pattern - get the ROWS first (tr), then loop through and get the TABLE CELLS (td)*

In [164]:
# get to the main line page
driver.get("https://arlweb.msha.gov/drs/drshome.htm")
input_field = driver.find_element_by_name("MineId")
input_field.send_keys('3901432')
search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
search_button.click()

In [165]:
date_field = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[2]/tbody/tr[2]/td/font/input[1]')
date_field.send_keys("1/1/1995")

In [166]:
violation_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input')
violation_button.click() 

In [167]:
reports_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input')
reports_button.click()

In [168]:
#Citation number
#Case number
#Standard violated
#Link to standard
#Proposed penalty
#Amount paid to date

In [169]:
from bs4 import BeautifulSoup

In [170]:
doc = BeautifulSoup(driver.page_source, 'html.parser')

In [171]:
violation_rows = doc.find_all('tr', class_='drsviols')
len(violation_rows)

18

In [172]:
for violation_row in violation_rows:
    print("I have a violation")
    cells = violation_row.find_all('td')
    print("Citation no", cells[2].text)
    print("Case no", cells[3].text)
    print("Standard violated", cells[10].find('a').text)
    print("Link", cells [10].find('a')['href'])
    print("Proposed penalty", cells[11].text)
    print("Amount paid to date", cells[14].text)
    
    print("I found", len(cells), "td elements")

    print("-----")

I have a violation
Citation no 8750964                        
Case no 000361866           
Standard violated 
56.18010            
Link http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-vol1/pdf/CFR-2014-title30-vol1-sec56-18010.pdf
Proposed penalty 100.00
Amount paid to date 100.00 
I found 15 td elements
-----
I have a violation
Citation no 6426439                        
Case no 000260865           
Standard violated 
56.4201(a)(2)       
Link http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-vol1/pdf/CFR-2011-title30-vol1-sec56-4201.pdf
Proposed penalty 100.00
Amount paid to date 100.00 
I found 15 td elements
-----
I have a violation
Citation no 6426438                        
Case no 000260865           
Standard violated 
56.4101             
Link http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-vol1/pdf/CFR-2011-title30-vol1-sec56-4101.pdf
Proposed penalty 100.00
Amount paid to date 100.00 
I found 15 td elements
-----
I have a violation
Citation no 6588189                        
Case n

In [173]:
#to save to a CSV
# 1. make an emtpy list
# 2. every time through the loop, create a dictionary of your data
# 3. add the dictionary to the list
# 4. when the entire loop is over, convert the list to a dataframe
# 5. save that dataframe

violations = []
for violation_row in violation_rows:
    violation = {}
    cells = violation_row.find_all('td')
    violation['citation'] = cells[2].text.strip()
    violation['case no'] = cells[3].text.strip()
    violation['standard'] = cells[10].find('a').text.strip()
    violation['link'] = cells [10].find('a')['href']
    violation['proposed'] = cells[11].text.strip()
    violation['paid'] = cells[14].text.strip()
    violations.append(violation)

In [174]:
violations

[{'case no': '000361866',
  'citation': '8750964',
  'link': 'http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-vol1/pdf/CFR-2014-title30-vol1-sec56-18010.pdf',
  'paid': '100.00',
  'proposed': '100.00',
  'standard': '56.18010'},
 {'case no': '000260865',
  'citation': '6426439',
  'link': 'http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-vol1/pdf/CFR-2011-title30-vol1-sec56-4201.pdf',
  'paid': '100.00',
  'proposed': '100.00',
  'standard': '56.4201(a)(2)'},
 {'case no': '000260865',
  'citation': '6426438',
  'link': 'http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-vol1/pdf/CFR-2011-title30-vol1-sec56-4101.pdf',
  'paid': '100.00',
  'proposed': '100.00',
  'standard': '56.4101'},
 {'case no': '000260865',
  'citation': '6588189',
  'link': 'http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-vol1/pdf/CFR-2011-title30-vol1-sec56-14200.pdf',
  'paid': '100.00',
  'proposed': '100.00',
  'standard': '56.14200'},
 {'case no': '000238554',
  'citation': '6588210',
  'link': 'http://www.gpo.gov/fdsys

In [175]:
import pandas as pd
df = pd.DataFrame(violations)
df.head()

,case no,citation,link,paid,proposed,standard
0,000361866,8750964,http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,100.00,100.00,56.18010
1,000260865,6426439,http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-...,100.00,100.00,56.4201(a)(2)
2,000260865,6426438,http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-...,100.00,100.00,56.4101
3,000260865,6588189,http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-...,100.00,100.00,56.14200
4,000238554,6588210,http://www.gpo.gov/fdsys/pkg/CFR-2010-title30-...,100.00,100.00,50.30(a)


In [176]:
df.to_csv("3901432-violations.csv", index=False)

In [177]:
df = pd.read_csv("mines-subset.csv", dtype={'id': 'str'})
df.head()

,id
0,4104757
1,0801306
2,3609931


# Using .apply to save mine data for SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

In [178]:
df = pd.read_csv("mines-subset.csv", dtype={'id': 'str'})
df.head()

,id
0,4104757
1,0801306
2,3609931


### Scrape the violations for each mine

**Save each mine's violations into separate CSV files.** Each CSV file must include the following fields:

- Citation number
- Case number
- Standard violated
- Link to standard
- Proposed penalty
- Amount paid to date

Make sure you are saving them into **separate files.** It might be nice to name them after the mine id.

- *TIP: Use .apply for this*
- *TIP: Print out the ID before you start scraping. That way you can take that ID and search manually to see if there is anything weird about the results.*
- *TIP: If you need help with .apply, look at the "Using apply in pandas" notebook 
- *TIP: It's probably worth it to print the fields first, then save them to a CSV once you know it's all working.*

In [190]:
def scrape_violations(row):
    print(row['id'])
    
    driver.get("https://arlweb.msha.gov/drs/drshome.htm")
    input_field = driver.find_element_by_name("MineId")
    input_field.send_keys(row['id'])
    search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
    search_button.click()
    
    
    # search for violations
    date_field = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[2]/tbody/tr[2]/td/font/input[1]')
    date_field.send_keys("1/1/1995")
    
    violation_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input')
    violation_button.click()
    
    reports_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input')
    reports_button.click()
    
    doc = BeautifulSoup(driver.page_source, 'html.parser')
    violation_rows = doc.find_all('tr', class_='drsviols')
    
    violations = []
    for violation_row in violation_rows:
        violation = {}
        cells = violation_row.find_all('td')
        violation['citation'] = cells[2].text.strip()
        violation['case no'] = cells[3].text.strip()
        a_tag = cells[10].find('a')
        if a_tag:
            violation['standard'] = a_tag.text.strip()
            violation['link'] = a_tag['href']
        if len(cells) > 14:
            violation['proposed'] = cells[11].text.strip()
            violation['paid'] = cells[14].text.strip()
        violations.append(violation)
            
    violations_df = pd.DataFrame(violations)
    violations_df.to_csv(row['id'] + "-violations.csv", index=False)
    

df.apply(scrape_violations, axis=1)

4104757
0801306
3609931


0    None
1    None
2    None
dtype: object

In [187]:
doc = BeautifulSoup(driver.page_source, 'html.parser')
violation_rows = doc.find_all('tr', class_='drsviols')
    
violations = []
for violation_row in violation_rows:
    violation = {}
    cells = violation_row.find_all('td')
    violation['citation'] = cells[2].text.strip()
    violation['case no'] = cells[3].text.strip()
    a_tag = cells[10].find('a')
    if a_tag:
        violation['standard'] = a_tag.text.strip()
        violation['link'] = a_tag['href']
    if len(cells) > 14:
        violation['proposed'] = cells[11].text.strip()
        violation['paid'] = cells[14].text.strip()
    violations.append(violation)

In [188]:
pd.DataFrame(violations)

,case no,citation,link,paid,proposed,standard
0,,8912694,http://www.ecfr.gov/cgi-bin/text-idx?SID=f462b...,NaN,NaN,56.14132(a)
1,000427623,8638781,http://www.gpo.gov/fdsys/pkg/CFR-2016-title30-...,351.00,351.00,56.12028
2,000411633,8903432,http://www.gpo.gov/fdsys/pkg/CFR-2016-title30-...,100.00,100.00,47.41(a)
3,000411633,8903433,http://www.gpo.gov/fdsys/pkg/CFR-2016-title30-...,100.00,100.00,56.14100(b)
4,000411633,8903434,http://www.gpo.gov/fdsys/pkg/CFR-2016-title30-...,117.00,117.00,46.11(d)
5,000411633,8903435,http://www.gpo.gov/fdsys/pkg/CFR-2016-title30-...,117.00,117.00,56.9300(a)
6,000411633,8903436,http://www.gpo.gov/fdsys/pkg/CFR-2016-title30-...,100.00,100.00,56.12004
7,000397972,8822977,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,100.00,100.00,56.9300(b)
8,000397972,8822975,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,100.00,100.00,56.14100(c)
9,000397972,8822973,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,100.00,100.00,56.9300(a)
